In [3]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import nltk
import re
from nltk.corpus import stopwords
from nltk.corpus import brown
from nltk import FreqDist
from nltk.corpus import stopwords
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.cross_validation import cross_val_score
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 9)
plt.style.use('ggplot')

In [46]:
#Importing complete and labled job summary data
complete_data = pd.read_csv('complete_labeled_job_summary_data.csv',encoding = "ISO-8859-1")
complete_data = complete_data.drop('Unnamed: 0',axis=1)

#converting text data into lower case letters in order to process better
complete_data["job summary"]=complete_data["job summary"].str.lower()

In [50]:
#Creating a combined list of all the words in job summary for training data

#Creating empty list for iterations and other for words in job summary
iterator = []
total_words_list=[]

#Creating a list with length equal to number of rows in the dataframe
for i in range (0,len(complete_data.axes[0]),1):
    iterator.append(i)
    
#creating the list
for i in iterator:
    job_summary=complete_data.get_value(i,'job summary')
    words = re.split(r"\s", job_summary)
    total_words_list.append(words)

#Combining all the nested arrays to form one single array    
total_words_list = [y for x in total_words_list for y in x]


In [54]:
#Create a string from the array of the words for training data
total_string = ' '.join(w for w in total_words_list)


In [57]:
#Convert string to nltk.text format for training data
tokens = nltk.word_tokenize(total_string)      # Convert str to token
nltk_words = nltk.text.Text(tokens)            # convert tokens to nltk.text.Text()


In [62]:
#Removing stop words
#identifying stopwords in english
stopwords = nltk.corpus.stopwords.words('english')

#Remove the stop words from list of words for training data
content = [w for w in nltk_words if w not in stopwords]


In [64]:
#Finding out features out of content for training data
#Finding out frequency of words
freq_words = nltk.FreqDist(w for w in content)

#Finding features
num_features = 20000
word_features = list(freq_words)[:num_features]
print(word_features)
type(word_features)


['splendor', 'villages.additional', 'regulations.identify', 'qi', 'teamnike', 'afis', 'unilever', 'parentcompany', 'groupsã¢â\x80â\x9d', 'lula', 'familyã¢â\x80â\x99s', '12xx', 'used', 'ska00071k', 'informationto', 'prospect', 'sl', 'attitude.must', 'provocative', 'mvr', 'provenability', '202,000', 'be-', 'management.5', 'specialist**', '-oral', 'cry', 'media.providing', '04333-0011', '_individuals', '2-2-3', 'fluorescent', 'nhrc', 'requestedcomfort', 'implement/', 'manner', 'eai/b2b', 'cart', 'experience.you', 'paulo.chagas', 'icuheart', 'qs', 'ilrequired', 'whowill', 'post-accident/incident', 'event.essential', 'activitiesmanage', 'shl', 'channelsmaintain', 'acvecc', 'empowerment.totals', '1t', 'www.tinyurl.com/mcd25697', 'circuitry', 'survivorship', 'regulatory/audit', 'counselorssupervise', 'yearscam', 'applies', 'prepare/maintain/create', 'monetary/unpaid', 'operations.estimates', 'marketpromotes', 'apply.attitude', 'guestsâ\x80\x99', 'plans/policies', 'assistantresponsibilitiesper

list

In [66]:
#Converting strings in train data into nltk.text.Text format in order to use NLP
#Creating empty list for iterations
iterator = []

#Creating a list with length equal to number of rows in the dataframe
for i in range (0,len(complete_data.axes[0]),1):
    iterator.append(i)

#Converting and replacing the strings with nltk.text    
for i in iterator:
    job_summary=complete_data.get_value(i,'job summary')  # Pull just the job summary as a str
    tokens = nltk.word_tokenize(job_summary)      # Convert str to token
    text_obj = nltk.text.Text(tokens)             # convert tokens to nltk.text.Text()
    complete_data.set_value(i,'job summary',text_obj)     # Now all of the functions from lab 17 will work with this obj



In [1]:
#Creating a list of tuples which combine words in job summary and its label
for i in range (0,len(complete_data.axes[0]),1):
    iterator.append(i)
    
total_job_listings_and_labels = []
for i in iterator:
    job_listings_and_labels = tuple([list(complete_data.get_value(0,"job summary")),
                                    complete_data.get_value(0,"label")])
    total_job_listings_and_labels.append(job_listings_and_labels)


NameError: name 'complete_data' is not defined

In [ ]:
def document_features(document):
    # Note: checking whether a word occurs in a set is much faster 
    # than checking whether it occurs in a list     
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

featuresets = [(document_features(d), c) for (d,c) in total_job_listings_and_labels]

In [ ]:
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

print(nltk.classify.accuracy(classifier, test_set))

classifier.show_most_informative_features(10)